In [50]:
import re
import pandas as pd
from collections import defaultdict
import numpy as np
from pathlib import PurePosixPath
from Bio import SeqIO

In [3]:
trinotateResults = pd.read_table("/Users/picciani/Dropbox/Yale/YCGA/podocoryna_analysis/reference/github-repo/trinotate_annotation_report.xls", index_col=0)

In [4]:
annGenes = trinotateResults.loc[(trinotateResults["sprot_Top_BLASTX_hit"] != ".") | (trinotateResults["sprot_Top_BLASTP_hit"] != ".")]
annGenes.shape #31,864 rows with annotation

(29335, 16)

In [5]:
annGenes['BLASTX_qstart'] = annGenes['sprot_Top_BLASTX_hit'].str.extract(".+\^Q:(\d+)-\d+,H:\d+-\d+\^", expand=False).str.strip()
annGenes['BLASTX_qend'] = annGenes['sprot_Top_BLASTX_hit'].str.extract(".+\^Q:\d+-(\d+),H:\d+-\d+\^", expand=False).str.strip()
annGenes['BLASTP_qstart'] = annGenes['sprot_Top_BLASTP_hit'].str.extract(".+\^Q:(\d+)-\d+,H:\d+-\d+\^", expand=False).str.strip()
annGenes['BLASTP_qend'] = annGenes['sprot_Top_BLASTP_hit'].str.extract(".+\^Q:\d+-(\d+),H:\d+-\d+\^", expand=False).str.strip()

/Users/picciani/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/picciani/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/picciani/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [21]:
annGenes["BLASTX_qstart"] = pd.to_numeric(annGenes["BLASTX_qstart"])
annGenes["BLASTX_qend"] = pd.to_numeric(annGenes["BLASTX_qend"])
annGenes["BLASTP_qstart"] = pd.to_numeric(annGenes["BLASTP_qstart"])
annGenes["BLASTP_qend"] = pd.to_numeric(annGenes["BLASTP_qend"])

/Users/picciani/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/picciani/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/picciani/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [20]:
annGenes.head(50)

,transcript_id,sprot_Top_BLASTX_hit,RNAMMER,prot_id,prot_coords,sprot_Top_BLASTP_hit,Pfam,SignalP,TmHMM,eggnog,...,gene_ontology_BLASTX,gene_ontology_BLASTP,gene_ontology_Pfam,transcript,peptide,BLASTX_qstart,BLASTX_qend,BLASTP_qstart,BLASTP_qend,is_reverse
#gene_id,,,,,,,,,,,,,,,,,,,,,
TRINITY_DN18_c0_g1,TRINITY_DN18_c0_g1_i1,"ELOC_RAT^ELOC_RAT^Q:467-138,H:3-112^87.273%ID^...",.,TRINITY_DN18_c0_g1_i1.p1,470-135[-],"ELOC_RAT^ELOC_RAT^Q:2-111,H:3-112^87.273%ID^E:...","PF03931.18^Skp1_POZ^Skp1 family, tetramerisati...",.,.,.,...,GO:0070449^cellular_component^elongin complex`...,GO:0070449^cellular_component^elongin complex`...,GO:0006511^biological_process^ubiquitin-depend...,.,.,467.0,138.0,2,111,True
TRINITY_DN95_c1_g1,TRINITY_DN95_c1_g1_i3,"S39AA_MOUSE^S39AA_MOUSE^Q:2266-884,H:301-806^3...",.,TRINITY_DN95_c1_g1_i3.p1,2530-818[-],"S39AA_MOUSE^S39AA_MOUSE^Q:89-567,H:301-824^36....",PF02535.25^Zip^ZIP Zinc transporter^195-563^E:...,sigP:1^21^0.729,.,.,...,GO:0005887^cellular_component^integral compone...,GO:0005887^cellular_component^integral compone...,GO:0046873^molecular_function^metal ion transm...,.,.,2266.0,884.0,89,567,True
TRINITY_DN85_c1_g1,TRINITY_DN85_c1_g1_i10,"ZN648_HUMAN^ZN648_HUMAN^Q:488-1375,H:281-529^2...",.,TRINITY_DN85_c1_g1_i10.p1,320-2107[+],"ZN648_HUMAN^ZN648_HUMAN^Q:57-352,H:281-529^25....",.,.,.,.,...,GO:0005634^cellular_component^nucleus`GO:00009...,GO:0005634^cellular_component^nucleus`GO:00009...,.,.,.,1016.0,1585.0,57,278,False
TRINITY_DN46_c0_g1,TRINITY_DN46_c0_g1_i2,"RTTN_HUMAN^RTTN_HUMAN^Q:3916-2165,H:890-1498^2...",.,TRINITY_DN46_c0_g1_i2.p1,3880-86[-],"RTTN_MOUSE^RTTN_MOUSE^Q:320-570,H:1247-1498^28...",.,.,.,.,...,GO:0005814^cellular_component^centriole`GO:000...,GO:0005814^cellular_component^centriole`GO:000...,.,.,.,1258.0,203.0,906,1226,True
TRINITY_DN17_c0_g1,TRINITY_DN17_c0_g1_i3,"B3GT9_HUMAN^B3GT9_HUMAN^Q:944-324,H:75-275^27....",.,TRINITY_DN17_c0_g1_i3.p1,1343-156[-],"B3GT9_HUMAN^B3GT9_HUMAN^Q:98-340,H:46-275^27.5...",PF01762.24^Galactosyl_T^Galactosyltransferase^...,.,.,.,...,GO:0005794^cellular_component^Golgi apparatus`...,GO:0005794^cellular_component^Golgi apparatus`...,GO:0016758^molecular_function^hexosyltransfera...,.,.,944.0,324.0,98,340,True
TRINITY_DN59_c1_g2,TRINITY_DN59_c1_g2_i3,"POLG_PVCV2^POLG_PVCV2^Q:2609-1659,H:1438-1742^...",.,TRINITY_DN59_c1_g2_i3.p1,4194-37[-],.,.,.,.,.,...,GO:0004190^molecular_function^aspartic-type en...,.,.,.,.,2609.0,1659.0,NaN,NaN,True
TRINITY_DN59_c1_g2,TRINITY_DN59_c1_g2_i3,"POLG_PVCV2^POLG_PVCV2^Q:2609-1659,H:1438-1742^...",.,TRINITY_DN59_c1_g2_i3.p2,3344-855[-],.,.,.,.,.,...,GO:0004190^molecular_function^aspartic-type en...,.,.,.,.,2609.0,1659.0,NaN,NaN,True
TRINITY_DN59_c1_g2,TRINITY_DN59_c1_g2_i3,"POLG_PVCV2^POLG_PVCV2^Q:2609-1659,H:1438-1742^...",.,TRINITY_DN59_c1_g2_i3.p3,371-3[-],.,.,.,.,.,...,GO:0004190^molecular_function^aspartic-type en...,.,.,.,.,2609.0,1659.0,NaN,NaN,True
TRINITY_DN59_c1_g4,TRINITY_DN59_c1_g4_i1,"POL_CSVMV^POL_CSVMV^Q:3105-1759,H:202-649^23.1...",.,TRINITY_DN59_c1_g4_i1.p1,4156-347[-],.,PF00589.25^Phage_integrase^Phage integrase fam...,.,.,.,...,GO:0004190^molecular_function^aspartic-type en...,.,GO:0003677^molecular_function^DNA binding`GO:0...,.,.,3105.0,1759.0,NaN,NaN,True


In [68]:
annGenes.to_csv("/Users/picciani/Dropbox/Yale/YCGA/podocoryna_analysis/reference/github-repo/trinotate_annotate_only_with_query_coord.cssuperv",header=True)

In [17]:
annGenes["is_reverse"] = np.where((annGenes['BLASTX_qstart'] > annGenes['BLASTX_qend']), "True", "False")

/Users/picciani/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [41]:
annGenesNeg = annGenes.loc[annGenes['BLASTX_qstart'] > annGenes['BLASTX_qend']]

In [42]:
annGenesNeg.to_csv("/Users/picciani/Dropbox/Yale/YCGA/podocoryna_analysis/reference/github-repo/trinotate_annotate_negative_with_query_coord.csv", header=True)

In [43]:
annGenesNeg.shape

(15551, 21)

In [44]:
annGenesNeg_transID = list(set(annGenesNeg["transcript_id"].tolist())) # convert column values to list, then find the set of non-duplicate values and convert that set to a list

In [47]:
outfile = "/Users/picciani/Dropbox/Yale/YCGA/podocoryna_analysis/reference/github-repo/trinotate_annotate_negative_transcript_ids.csv"

with open (outfile, "w") as out:
    for i in annGenesNeg_transID:
        out.write(i + "\n")

In [51]:
transcriptomePath = "/Users/picciani/Dropbox/Yale/YCGA/podocoryna_analysis/reference/Podocoryna_carnea_transcriptome_clean_2021.fasta_longestORFperGene.fasta"
output = "/Users/picciani/Dropbox/Yale/YCGA/podocoryna_analysis/reference/github-repo"

transcriptomeName = PurePosixPath(transcriptomePath).stem
outputPath = output + "/" + transcriptomeName + ".allpositive.fasta"

# Replaces records in list of negatives with their reverse complement
with open (transcriptomePath) as transcriptome:
    with open (outputPath,'w') as outfile:
        records = SeqIO.to_dict(SeqIO.parse(transcriptome, "fasta")) #parse transcriptome as dictionary {record.id : SeqRecord}
        for item in annGenesNeg_transID: #for record ids in list of negatives
            records[item]=records[item].reverse_complement(id=True, name=True, description = True) #replace their SeqRecord with reverse complement
        SeqIO.write(records.values(),outfile,"fasta") #write dict values to fastafile